# Hello world
In this notebook, you'll run a minimal interesting example of `mandala`. It's a
great way to quickly get a feel for the library and play with it yourself!

If you want a more in-depth introduction with a real ML project, check out the
[the next tutorial](01_logistic.ipynb).

## Create the storage and computational primitives
A `Storage` instance is where the results of all computations you run in a
project are stored. Importantly, **the only way to put data into a `Storage` is
to call a function**: you decorate your functions with the `@op` decorator, and
then any time you call them, the inputs and outputs for this call are stored in
the `Storage`.

Go ahead and create a storage and two `@op`-decorated functions:

In [1]:
from mandala.imports import *

# create a storage for results
storage = Storage()

@op # memoization decorator
def inc(x) -> int:
    print('Hi from inc!')
    return x + 1 

@op
def add(x: int, y: int) -> int:
    print('Hi from add!')
    return x + y

### A note on function inputs/outputs
Currently, **`mandala` only supports functions with a fixed number of inputs and
outputs**. To help make this explicit for outputs, you must specify
the number of outputs in the return type annotation. For example, `def f() ->
int` means that `f` returns a single integer, and `def f() -> Tuple[int, int]` means
that `f` returns two integers. Functions that return nothing can leave the
return type annotation empty.

## Your first `mandala`-tracked computation
The main way you use `mandala` is through "workflows", i.e. compositions of
`@op`-decorated functions. Running a workflow for the first time inside a
`storage.run()` block will execute the workflow and store the results in the 
`storage`:

In [2]:
with storage.run():
    x = inc(20)
    y = add(21, x)
    print(y)

Hi from inc!
Hi from add!
ValueRef(42, uid=d92...)


Running this workflow a **second** time will not re-execute it, but instead
retrieve the results from the `storage` at each function call along the way:

In [3]:
with storage.run():
    x = inc(20)
    y = add(21, x)
    print(y)

ValueRef(42, uid=d92...)


Adding more logic to the workflow will not re-execute the parts that have
already been executed:

In [4]:
with storage.run():
    for a in [10, 20, 30]:
        x = inc(a)
        y = add(21, x)

Hi from inc!
Hi from add!
Hi from inc!
Hi from add!


The workflow just executed can also be used as a jumping-off point for issuing
queries. For example, `storage.similar(...)` can be used to query for a table of
values that were computed in an analogous way to given variables:

In [5]:
storage.similar(y, context=True) # use `context=True` to also get the values of dependencies

Pattern-matching to the following computational graph (all constraints apply):
    a0 = Q() # input to computation; can match anything
    x = inc(x=a0)
    a1 = Q() # input to computation; can match anything
    y = add(x=a1, y=x)
    result = storage.df(a0, x, a1, y)


,a0,x,a1,y
1,10,11,21,32
0,20,21,21,42
2,30,31,21,52


The `storage.similar` method prints out the query extracted from the
computation. For more control (or if you dislike how implicit the interface
above is), you can directly copy-paste this code into a `storage.query()` block:

In [6]:
with storage.query():
    a = Q() # input to computation; can match anything
    a0 = Q() # input to computation; can match anything
    x = inc(x=a)
    y = add(x=a0, y=x)
storage.df(a, a0, x, y)

Pattern-matching to the following computational graph (all constraints apply):
    a = Q() # input to computation; can match anything
    x = inc(x=a)
    a0 = Q() # input to computation; can match anything
    y = add(x=a0, y=x)
    result = storage.df(a, a0, x, y)


,a,a0,x,y
1,10,21,11,32
0,20,21,21,42
2,30,21,31,52


Those are the main patterns you need to know to start playing around with
`mandala`! We invite you to go back and modify the code above by creating new
computational primitives and workflows, and see how `mandala` handles it.

TODO: talk about versioning!